In [1]:
#from huggingface_hub.utils import disable_progress_bars
#from huggingface_hub import snapshot_download

# NOTICE: HF dataset is utilized as an archive of the original data, the following code downloads the 
# dataset to the local machine

#disable_progress_bars()
#
#REPO_ID = "sodowo/documents"
#
#snapshot_download(repo_id=REPO_ID, repo_type="dataset", local_dir="./documents")


In [3]:
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q -U peft
%pip install -q -U accelerate
%pip install -q -U wandb
%pip install -q -U datasets
%pip install -q -U pdf2image
%pip install -q -U python-dotenv
!apt-get update --yes
!apt-get install -y poppler-utils

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Get:1 file:/var/cuda-repo-ubuntu2204-12-4-local  InRelease [1572 B]
Get:2 file:/var/cudnn-local-repo-ubuntu2204-9.5.1  InRelease [1572 B]
Get:1 file:/var/cuda-repo-ubuntu2204-12-4-local  InRelease [1572 B]      
Get:2 file:/var/cudnn-local-repo-ubuntu2204-9.5.1  InRelease [1572 B]          
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:5 http://archive

In [1]:
import torch
import random
import wandb
import os
import unicodedata
from transformers import DonutProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision as tv
from IPython.display import Image
from tqdm.auto import tqdm
from pdf2image import convert_from_path
from datasets import load_dataset
from dotenv import load_dotenv
from datetime import datetime

In [2]:
load_dotenv()

WEIGHTS_AND_BIASES = os.environ["WEIGHTS_AND_BIASES"]
HF_TOKEN = os.environ["HF_TOKEN"]

In [3]:
dataset = load_dataset("sodowo/doc_meta")["train"]

In [4]:
dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1969x1386>,
 'additional_features': {'title': 'Entschuldigung',
  'date': None,
  'content_type': 'sonstiges',
  'has_signature': False,
  'main_author': {'type': 'company',
   'email': None,
   'phone': None,
   'fax': None,
   'address': None,
   'website': None,
   'name': 'OLYMPIA Schnittmusterverlag',
   'sector': None,
   'first_name': None,
   'last_name': None,
   'role': None,
   'mobile': None,
   'department': None,
   'state': None,
   'district': None,
   'city': None},
  'other_authors': None,
  'logo_owners': ['OLYMPIA Schnittmusterverlag'],
  'recipients': None}}

In [5]:
class AuthorExtractionDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.task_start_token = self.eos_token = "<s>"
        self.processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
        self.processor.tokenizer.add_special_tokens(
            {"additional_special_tokens": [self.task_start_token, self.eos_token]}
        )
        self.processor.feature_extractor.do_align_long_axis = False
        self.processor.feature_extractor.size = [1920, 2560]
        self.data = self._preprocess_data()

    def _preprocess_data(self):
        data = []
        for idx, elem in tqdm(enumerate(self.dataset), total=len(self.dataset), desc="Preparing dataset"):

            additional_features = elem["additional_features"]

            main_author = additional_features["main_author"]

            if main_author is None:
                continue

            entity_type = main_author["type"]

            author_string = ""

            match entity_type:
                case "person":
                    if main_author["first_name"] is not None:
                        author_string += main_author["first_name"]
                    if main_author["last_name"] is not None:
                        author_string += " " + main_author["last_name"]
                
                case "government":
                    author_string += main_author["department"]
                
                case "company" | "organization" | "group":
                    author_string += main_author["name"]
                
                case _:
                    continue

            author_string = author_string.strip()

            author = self.task_start_token + author_string + self.eos_token

            # remove empty author strings
            if author.replace(" ", "") == "<s></s>":
                continue

            # remove author strings that are too long
            if len(author) > 80:
                continue

            data.append((idx, author))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        idx, author = self.data[idx]

        image_data = self.dataset[idx]["image"]

        pixel_values = self.processor(image_data, return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        input_ids_author = (
            self.processor.tokenizer(
                author,
                add_special_tokens=False,
                padding="max_length",
                truncation=True,
                max_length=80,
                return_tensors="pt",
            )["input_ids"]
            .squeeze(0)
            .long()
        )

        author_target = input_ids_author.clone()
        author_target[author_target == self.processor.tokenizer.pad_token_id] = -100
        return {
            "pixel": pixel_values,
            "target": author_target,
        }

    def get_split(self, train_ratio=0.8, val_ratio=0.1, seed=None):
        total_size = len(self.data)
        train_size = int(train_ratio * total_size)
        val_size = int(val_ratio * total_size)
        test_size = total_size - train_size - val_size  # Ensure all data is used

        indices = list(range(total_size))
        if seed is not None:
            random.Random(seed).shuffle(indices)
        else:
            random.shuffle(indices)

        train_indices = indices[:train_size]
        val_indices = indices[train_size : train_size + val_size]
        test_indices = indices[train_size + val_size :]

        train_dataset = Subset(self, train_indices)
        val_dataset = Subset(self, val_indices)
        test_dataset = Subset(self, test_indices)

        return train_dataset, val_dataset, test_dataset

In [6]:
author_dataset = AuthorExtractionDataset(dataset)

/usr/local/lib/python3.11/dist-packages/transformers/models/donut/processing_donut.py:208: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


Preparing dataset:   0%|          | 0/4702 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (139521375 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (90675003 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (138720000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [7]:
train, val, test = author_dataset.get_split(0.8, 0.18)
len(train), len(val), len(test)

(3541, 796, 90)

In [8]:
batch_size = 1
num_workers = 0

train_dataloader = DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True
)

val_dataloader = DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

test_dataloader = DataLoader(
    test, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

In [45]:
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

model.decoder.resize_token_embeddings(len(author_dataset.processor.tokenizer))
model.config.pad_token_id = author_dataset.processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = author_dataset.processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]
model.config.encoder.image_size = author_dataset.processor.feature_extractor.size[::-1] # (height, width)
model.config.decoder.max_length = 80

model = model.to("cuda")

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    2560,
    1920
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.46.1",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add

In [46]:
model.config

VisionEncoderDecoderConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "naver-clova-ix/donut-base",
  "architectures": [
    "VisionEncoderDecoderModel"
  ],
  "decoder": {
    "_attn_implementation_autoset": false,
    "_name_or_path": "",
    "activation_dropout": 0.0,
    "activation_function": "gelu",
    "add_cross_attention": true,
    "add_final_layer_norm": true,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 1024,
    "decoder_attention_heads": 16,
    "decoder_ffn_dim": 4096,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 4,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "encoder_layer

In [47]:
torch.set_float32_matmul_precision("medium")

In [48]:
wandb.login(key=WEIGHTS_AND_BIASES)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Training

In [49]:
class TrainingWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, pixel_values, labels):
        outputs = self.model(
            pixel_values=pixel_values,
            labels=labels
        )
        return outputs.loss, outputs.logits

    def generate(self, pixel_values):
        return self.model.generate(pixel_values=pixel_values)

train_model = TrainingWrapper(model).to("cuda")

In [50]:
optimizer = torch.optim.AdamW(train_model.parameters(), lr=1e-4)

accumulate_steps = 5


run = wandb.init(
    project="donut_torch",
    config={
        "learning_rate": 1e-4,
        "accumulation_steps": accumulate_steps,
        "batch_size": batch_size,
    }
)


def train_one_epoch(epoch_index):
    running_loss = 0.0
    last_loss = 0.0
    train_model.train()
    
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    
    for i, batch in progress_bar:
        # Move batch to GPU and use mixed precision
        pixel_values = batch["pixel"].to("cuda")
        labels = batch["target"].to("cuda")
        
        loss, _ = train_model(pixel_values=pixel_values, labels=labels)
        loss = loss / accumulate_steps
        
        loss.backward()
        
        # Gradient accumulation
        if (i + 1) % accumulate_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            
            # Log metrics
            last_loss = running_loss / accumulate_steps
            progress_bar.set_description(f"Loss: {last_loss:.4f}")
            wandb.log({
                "train_loss": last_loss
            })
            running_loss = 0.0
        else:
            running_loss += loss.item() * accumulate_steps

        # Evaluate on test set periodically
        if i % 200 == 190:
            test_loss = evaluate_model(test_dataloader)
            wandb.log({
                "test_loss": test_loss
            })
            train_model.train()  # Switch back to training mode

    return last_loss

def evaluate_model(dataloader):
    train_model.eval()
    total_loss = 0
    num_batches = 0
    
    with torch.no_grad():
        for batch in dataloader:
            pixel_values = batch["pixel"].to("cuda")
            labels = batch["target"].to("cuda")
            
            with torch.amp.autocast("cuda"):
                loss, _ = train_model(pixel_values=pixel_values, labels=labels)
            
            total_loss += loss.item()
            num_batches += 1
    
    return total_loss / num_batches

test_loss,▁
train_loss,█▄▂▁
test_loss,4.85931
train_loss,5.1474


In [51]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.0

for epoch in range(EPOCHS):
    print("EPOCH {}:".format(epoch_number + 1))

    train_model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    running_vloss = 0.0

    train_model.eval()

    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            vbatch = vdata
            with torch.amp.autocast("cuda"):
                loss, _ = train_model(
                    pixel_values=vbatch["pixel"].to("cuda"),
                    labels=vbatch["target"].to("cuda"),
                )
                vloss = loss
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))
    wandb.log({"val_loss": avg_vloss})

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        print("Saving model ...")
        best_vloss = avg_vloss
        model_path = "model_{}_{}".format(timestamp, epoch_number)
        torch.save(train_model.state_dict(), model_path)
        wandb.save(model_path)

    epoch_number += 1

EPOCH 1:


  0%|          | 0/3541 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (138720000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (139521375 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (90675003 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


LOSS train 1.0703526362776756 valid 1.6220632791519165
Saving model ...
EPOCH 2:


  0%|          | 0/3541 [00:00<?, ?it/s]

LOSS train 1.3687880150973797 valid 1.5589832067489624
Saving model ...
EPOCH 3:


  0%|          | 0/3541 [00:00<?, ?it/s]

LOSS train 0.40083192265592515 valid 1.648578405380249
EPOCH 4:


  0%|          | 0/3541 [00:00<?, ?it/s]

: 

In [37]:
torch.save(train_model, "model.pt")

wandb: 🚀 View run legendary-lotus-110 at: https://wandb.ai/jonasg/donut_torch/runs/tkeax217
wandb: Find logs at: wandb/run-20241101_225717-tkeax217/logs


In [19]:
train_model = torch.load("model.pt")

/tmp/ipykernel_1109/882432359.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_model = torch.load("model.pt")


In [18]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [21]:
train_model.model.push_to_hub("doc_meta")

ImportError: cannot import name 'TensorParallelOcShardConv2d' from 'deepspeed.module_inject.layers' (/usr/local/lib/python3.11/dist-packages/deepspeed/module_inject/layers.py)